In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

from bs4 import BeautifulSoup
import requests
import csv
from random import randint
from time import sleep

In [2]:
df = pd.DataFrame()
with open('links.csv', newline='') as f:
    reader = csv.reader(f)
    your_list = list(reader)
links = []
for i in range(len(your_list)):
    links.append(your_list[i][0])
links = list(set(links))

In [3]:
def date_parse(script, day1, day2):
    parse = script[script.index(day1) + len(day1) + 2 : script.index(day2)-2]
    return parse.replace('[','').replace(']','').replace('false', "0").replace('true', "1").replace(',','')

def date_add(date_str):
        count = 0
        for i in date_str:
            count += int(i)
        return count
    
def make_series(value, series_name):
    return pd.Series(value, name=series_name)

In [4]:
def scrap_url(url):
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html')
    soup_text = soup.prettify()
    
    try:
        description = soup.find('p', {"class":"spc-tiny medium-hide secondary-text light-header"}).text
    except:
        description = ''
    
    try:
        hour_str = soup.find('h3').text
        hours_tutor = int(hour_str[:hour_str.index(' hours')].replace(',',""))
    except: 
        hours_tutor = ''
    try:
        rate_str = soup.find('h4').div.contents[0]
        rate = int(rate_str[rate_str.index('$')+1:])
    except: 
        rate = ''
    
    try:   
        bio = ''
        for tag in soup.find_all('div', {"class":"full-content hide"}):
            bio = bio + tag.text
            bio = bio.replace('\n',"")
    except:
        bio = ''
    
    try:
        edu = []
        for tag in soup.find_all('section', {"class":"spc-med-s"}):
            item = str(tag.text).replace('\n', '').strip()
            item = " ".join(item.split())
            edu.append(item)
    except:
        edu = ''
        
    try:
        subjects_emph = set()
        for i in soup.find_all('a', {"class":"text-nowrap"}):
            item = str(i.text).replace('\n', '').strip()
            item = item.replace(',', '')
            item = " ".join(item.split())
            subjects_emph.add(item)
        subjects_emph.discard('Google Play')
        subjects_emph.discard('App Store')
        
    except:
        subjects_emph = ''
        
    try:
        subjects_all = set()
        for i in soup.find_all('span', {'class':'text-nowrap'}):
            item = str(i.text).replace('\n', '').strip()
            item = item.replace(',', '')
            item = " ".join(item.split())
            subjects_all.add(item)
    except:
        subjects_all = ''
        
    try:
        ratings_text = soup.find('a',{"class":"ratings-anchor"}).text.replace('(','').replace(')','').replace(',', '')
        ratings_split = ratings_text.split()
        num_ratings = int(ratings_split[0])
        num_unique_students = hours_tutor / num_ratings
    except:
        ratings_text = ''
        num_ratings = ''
        num_unique_students = ''
           
    try:
        cal = soup.find('div',{"class":"availabilities-container hide"}).script.text
        sun = date_parse(cal, 'sunday', 'monday')
        mon = date_parse(cal, 'monday', 'tuesday')
        tues = date_parse(cal, 'tuesday', 'wednesday')
        wed = date_parse(cal, 'wednesday', 'thursday')
        thurs = date_parse(cal, 'thursday', 'friday')
        fri = date_parse(cal, 'friday', 'saturday')
        sat = cal[cal.index('saturday') + 10 : -2].replace('[','').replace(']','').replace('false', "0").replace('true', "1").replace(',','')
        hours_sun = date_add(sun)
        hours_mon = date_add(mon)
        hours_tues = date_add(wed)
        hours_wed = date_add(wed)
        hours_thurs = date_add(thurs)
        hours_fri = date_add(fri)
        hours_sat = date_add(sat)
        hours_per_week_schedule = hours_sun + hours_mon + hours_tues + hours_wed + hours_thurs + hours_fri + hours_sat
        
    except:
        hours_sun = ''
        hours_mon = ''
        hours_tues = ''
        hours_wed = ''
        hours_thurs = ''
        hours_fri = ''
        hours_sat = ''
        hours_per_week_schedule = ''
        
    try:    
        bg_check = soup.find_all('p',{"class":'spc-zero'})[1].text
        bg_check_date = bg_check.replace('\n', '').split()[4]
        bg_check_date = datetime.strptime(bg_check_date, '%m/%d/%Y').date()
    
    except:
        bg_check_date = ''
    
    try:
        review_dates = []
        for i in range(len(soup_text)):
            if soup_text[i:i+11] == 'dateCreated':
                review_dates.append(soup_text[i:i+24])
    except:
        review_dates = ''
        
    descriptions = pd.Series(description, name='descriptions')
    hours_tutor = pd.Series(hours_tutor, name='hours_tutor')
    rate = pd.Series(rate, name='rate')
    bio = pd.Series(bio, name='bio')
    edu = pd.Series([edu], name='edu')
    subjects_emph = pd.Series([list(subjects_emph)], name='subjects_emph')
    subjects_all = pd.Series([list(subjects_all)], name='subjects_all')
    num_ratings = pd.Series(num_ratings, name='num_ratings')
    hours_sun = pd.Series(hours_sun, name='hours_sun')
    hours_mon = pd.Series(hours_mon, name='hours_mon')
    hours_tues = pd.Series(hours_tues, name='hours_tues')
    hours_wed = pd.Series(hours_wed, name='hours_wed')
    hours_thurs = pd.Series(hours_thurs, name='hours_thurs')
    hours_fri = pd.Series(hours_fri, name='hours_fri')
    hours_sat = pd.Series(hours_sat, name='hours_sat')
    hours_per_week_schedule = pd.Series(hours_per_week_schedule, name='hours_per_week_schedule')
    bg_check_date = pd.Series(bg_check_date, name='bg_check_date')
    review_dates = pd.Series([list(review_dates)], name='review_dates')
    
    scraped_data = pd.concat([descriptions, hours_tutor, rate, bio, num_ratings, subjects_emph, subjects_all, edu,
                       hours_sun, hours_mon, hours_tues, hours_wed, hours_thurs, hours_fri, hours_sat,
                       hours_per_week_schedule, bg_check_date, review_dates], axis=1)     
    return scraped_data

In [72]:
%%time
for i in range(10):
#for i in range(len(links)):
    scraped = scrap_url(links[i])
    df = df.append(scraped)
    sleep(randint(1,5))
#df = df.reset_index(drop=True)
#df.to_csv('test_tutor_data.csv')
df_full = df.reset_index(drop=True)
#df_full.to_csv('full_tutor_data.csv')

CPU times: user 2.23 s, sys: 54 ms, total: 2.28 s
Wall time: 35.2 s


In [79]:
for i in df_full.review_dates:
    print(i)

['dateCreated":"2020-01-18', 'dateCreated":"2020-02-28', 'dateCreated":"2020-04-30', 'dateCreated":"2020-05-10', 'dateCreated":"2020-05-21']
['dateCreated":"2017-09-01', 'dateCreated":"2017-10-19', 'dateCreated":"2017-11-01', 'dateCreated":"2017-11-03', 'dateCreated":"2018-02-06', 'dateCreated":"2018-12-21']
['dateCreated":"2020-01-17', 'dateCreated":"2020-01-27', 'dateCreated":"2020-03-24']
['dateCreated":"2016-05-19', 'dateCreated":"2016-04-02', 'dateCreated":"2016-03-29', 'dateCreated":"2016-02-16', 'dateCreated":"2017-10-14', 'dateCreated":"2018-04-15', 'dateCreated":"2018-09-30']
[]
['dateCreated":"2020-01-18', 'dateCreated":"2020-02-28', 'dateCreated":"2020-04-30', 'dateCreated":"2020-05-10', 'dateCreated":"2020-05-21']
['dateCreated":"2017-09-01', 'dateCreated":"2017-10-19', 'dateCreated":"2017-11-01', 'dateCreated":"2017-11-03', 'dateCreated":"2018-02-06', 'dateCreated":"2018-12-21']
['dateCreated":"2020-01-17', 'dateCreated":"2020-01-27', 'dateCreated":"2020-03-24']
['dateCrea

In [8]:
url = 'https://www.wyzant.com/match/tutor/86182268?search_id=10258f52-7036-483d-87d3-7852f0c963c2'

c = []
for i in df_url
for i in scrap_url(url).review_dates[0]:
    c.append(datetime.strptime(i[14:], '%Y-%m-%d').date())
df_url['earliest_review'] = min(c)
df_url['latest_review'] = max(c)
    
today = date.today()
days_since_earliest_review = (today - df_url['earliest_review'][0]).days
days_since_latest_review = (today - df_url['latest_review'][0]).days
df_url['hours_per_week_since_earliest_review'] = (df_url['hours_tutor'] / days_since_earliest_review) * 7
    
cols_to_drop = ['review_dates', '']
df_url.drop(columns=cols_to_drop)

In [25]:
c = []
for i in scrap_url(url).review_dates[0]:
    c.append(datetime.strptime(i[14:], '%Y-%m-%d').date())
min(c)
df_url['earliest_review'] = min(c)
df_url['latest_review'] = max(c)

In [50]:
today = date.today()
df_url['days_since_earliest_review'] = (today - df_url['earliest_review'][0]).days
df_url['days_since_latest_review'] = (today - df_url['latest_review'][0]).days
df_url['hours_per_week_since_earliest_review'] = (df_url['hours_tutor'] / df_url['days_since_earliest_review']) * 7

In [69]:
df_url['description_length'] = len(df_url['descriptions'][0].split())
df_url['bio_length'] = len(df_url['bio'][0].split())
df_url['number_subjects'] = len(df_url['subjects_emph'][0]) + len(df_url['subjects_all'][0])
df_url['number_degrees'] = len(df_url['edu'][0])

In [70]:
df_url

,descriptions,hours_tutor,rate,bio,num_ratings,subjects_emph,subjects_all,edu,hours_sun,hours_mon,...,review_dates,earliest_review,latest_review,days_since_earliest_review,days_since_latest_review,hours_per_week_since_earliest_review,description_length,bio_length,number_subjects,number_degrees
0,"Expert mathematician, Functional programmer, T...",239,79,*** Me ***Mathematician turned software (block...,77,"[Logic, Discrete Math, Computer Programming, C...","[Algebra 2, Prealgebra, Algebra 1, Precalculus...","[FAU Honors College Mathematics and physics, K...",5,10,...,"[dateCreated"":""2020-03-28, dateCreated"":""2020-...",2020-03-28,2020-05-28,67,6,24.970149,7,287,13,3
